In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import seaborn as sns

In [ ]:
df = pd.read_pickle("accidents.pkl.gz")
df

# Hypotéza 1:

Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako dálnicích.

K ověření hypotézy využijte 𝜒<sup>2</sup> test s tím, že také určíte, jestli nehody na silnicích 1. třídy
vedly častěji či méně často k fatální nehodě. V tom vám může pomoct “expected” výstup 𝜒<sup>2</sup>
testu.

Nerozlišujte počet následků, ale pouze nehody, kde došlo k jednomu a více úmrtí
(p13a > 0).


In [ ]:
# Výbět nehod, kde došlo k jednomu nebo více úmrtí
dfh1 = df[df["p13a"] > 0].copy()

# Výběr silnic 1. třídy do nového dataframe
dfh1_1stclass = dfh1[dfh1["p36"] == 1].copy()

# Výběr dálnic do nového dataframe
dfh1_highway = dfh1[dfh1["p36"] == 0]

display(dfh1_1stclass)
display(dfh1_highway)